# Extracción de datos

En esta sección se extraerá los datos de la URL citada a continuación. Los indicadores tomados en cuenta son: 
* Edad de la población
* Renta Media
* Niveles de Educación
* Extranjeros
* Tamaño medio del Hogar
* Número de Desempleados
> https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=71359583a773a510VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default*

In [53]:
import pandas as pd

In [54]:
# Función de Lectura de barrios de Madrid
def map_barrios(dfOtro, dfBName):
    dff = dfOtro.merge(dfBName, on='BARRIOS', how='left')
    return dff


dfBarrios = pd.read_excel('../../original_data/Barrios.xls')
dfBarrios = dfBarrios[['XCoord', 'YCoord', 'BARRIO_MAY', 'COD_DISBAR']]
dfBarrios.columns = ['LONGITUD', 'LATITUD', 'BARRIOS' ,'COD_DISBAR']
# dfBarrios = dfBarrios.rename(columns = {'BARRIO_MAY': 'BARRIOS'}, inplace = False)
dfBarrios = dfBarrios.T.groupby(level=0).first().T    
dfBarrios

,BARRIOS,COD_DISBAR,LATITUD,LONGITUD
0,SAN CRISTÓBAL,172,4465820.907,441533.8022
1,BUTARQUE,173,4465394.096,442559.7779
2,LOS ÁNGELES,175,4467482.211,440632.5318
3,LOS ROSALES,174,4467464.76,441536.5956
4,"VILLAVERDE ALTO, CASCO HISTÓRICO DE VILLAVERDE",171,4465896.736,439786.7051
...,...,...,...,...
126,VALVERDE,86,4483501.173,442483.6118
127,EL GOLOSO,88,4487307.227,440716.0024
128,EL PARDO,81,4489409.578,435018.7103
129,CASCO HISTÓRICO DE VICÁLVARO,191,4471000.988,450789.392


In [55]:
# Función Genérica para iterar fuente 
# Lectura iterativa de excel
def iteracion_callback(contYear, indicadorName, contadorInicial, contadorFinal, unidad): 
    sheetNamesLst=['CENTRO', 'ARGANZUELA', 'RETIRO', 'SALAMANCA', 'CHAMARTÍN', 'TETUÁN', 'CHAMBERÍ', 'FUENCARRAL-EL PARDO','MONCLOA-ARAVACA', 'LATINA', 'CARABANCHEL', 'USERA', 'PUENTE DE VALLECAS', 'MORATALAZ', 'CIUDAD LINEAL', 'HORTALEZA', 'VILLAVERDE', 'VILLA DE VALLECAS', 'VICÁLVARO','SAN BLAS-CANILLEJAS','BARAJAS']
    arrayFullEdad = []
    dfPreProcesadoFull = pd.DataFrame()
    # Itera Paginas del excel
    for x, nameSheet in enumerate(sheetNamesLst):   
        dfAux = pd.DataFrame()
        df2 = pd.DataFrame()        
        #Lectura de excel
        fileName = ''
        fileName = '../../original_data/panel_indicadores_distritos_barrios_' + str(contYear) + '.xls'
        df = pd.read_excel(fileName, 
                           sheet_name=nameSheet, 
                           skiprows= 1)
        # Se posiciona en filas de edad en caso de la 
        # última página seteamos el número de columnas a leer por existir 
        # más columnas basura
        if x == 20 :
            dfAux=df.iloc[contadorInicial:contadorInicial+contadorFinal, 0:16]            
        else:
            dfAux=df.iloc[contadorInicial:contadorInicial+contadorFinal]            

        dfAux.reset_index(inplace=True, drop=True)
        
        columnasNombre = []
        contadorFor=0
        # Itera columnas para controlar celdas combinadas
        for col in dfAux.columns:
            if contadorFor == 0 :
                newCol = indicadorName
            elif contadorFor % 2 == 0:
                newCol = nomColAux.upper()
            else :
                newCol = 'GT_' + col
            columnasNombre.append(newCol)
            nomColAux = col
            contadorFor = contadorFor + 1
        # Se coloca los nuevos nombres de columnas
        dfAux.columns = columnasNombre
        
        indexColumnasBorrar = []
        contadorColumnas = 0
        # Obtener indices de las columnas para ser borradas las de porcentaje
        for row in columnasNombre:
            if contadorColumnas % 2 != 0 :
                indexColumnasBorrar.append(contadorColumnas)
            contadorColumnas = contadorColumnas + 1
        dfAux.drop(dfAux.columns[indexColumnasBorrar], axis = 1, inplace=True)
        dfAux.drop(dfAux.columns[[1]], axis = 1, inplace=True)
        
        # Pasar columnas como valores del dataframe bajo el campo Barrios
        df2 = pd.melt(dfAux, id_vars=[indicadorName], 
                      var_name="BARRIOS", value_name=unidad)
        # Strings a mayúsculas
        df2.columns = df2.columns.str.upper()
        # Colocamos el anio correspodniente
        df2['ANIO'] = contYear
        # Reemplazamos barrios mal escritos
        df2.loc[df2['BARRIOS'] == 'PACIFICO', 'BARRIOS'] = 'PACÍFICO'
        df2.loc[df2['BARRIOS'] == 'RIOS ROSAS', 'BARRIOS'] = 'RÍOS ROSAS'
        df2.loc[df2['BARRIOS'] == 'VALDEACEREDAS', 'BARRIOS'] = 'VALDEACEDERAS'
        df2.loc[df2['BARRIOS'] == 'FUENTELARREINA', 'BARRIOS'] = 'FUENTELAREINA'
        df2.loc[df2['BARRIOS'] == 'EL PILAR', 'BARRIOS'] = 'PILAR'
        df2.loc[df2['BARRIOS'] == 'CIUDAD \nUNIVERSITARIA', 'BARRIOS'] = 'CIUDAD UNIVERSITARIA'
        df2.loc[df2['BARRIOS'] == 'PALOMERAS \nSURESTE', 'BARRIOS'] = 'PALOMERAS SURESTE'
        df2.loc[df2['BARRIOS'] == 'VENTAS ', 'BARRIOS'] = 'VENTAS'
        df2.loc[df2['BARRIOS'] == 'SAN JUAN \nBAUTISTA', 'BARRIOS'] = 'SAN JUAN BAUTISTA'
        df2.loc[df2['BARRIOS'] == 'APÓSTOLO SANTIAGO', 'BARRIOS'] = 'APÓSTOL SANTIAGO'
        df2.loc[df2['BARRIOS'] == 'CASCO HISTÓRICO DE\nVALLECAS', 'BARRIOS'] = 'CASCO HISTÓRICO DE VALLECAS'
        df2.loc[df2['BARRIOS'] == 'CASCO HISTÓRICO DE\nVICÁLVARO', 'BARRIOS'] = 'CASCO HISTÓRICO DE VICÁLVARO'
        df2.loc[df2['BARRIOS'] == 'CASCO HISTÓRICO\nDE BARAJAS', 'BARRIOS'] = 'CASCO HISTÓRICO DE BARAJAS'
        df2.loc[df2['BARRIOS'] == 'TIMÓN ', 'BARRIOS'] = 'TIMÓN'      
        df2.loc[df2['BARRIOS'] == 'VILLAVERDE ALTO, CASCO HISTORICO DE VILLAVERDE ', 'BARRIOS'] = 'VILLAVERDE ALTO, CASCO HISTÓRICO DE VILLAVERDE'
        
        arrayFullEdad.append(df2)
        
    dfPreProcesadoFull = pd.concat(arrayFullEdad)                
    
    return dfPreProcesadoFull

## Edad de la población periodo 2016, 2018, 2019

In [56]:
# Dataset Nivel 1
anio_indicador = 2016
dfEdad2016 = iteracion_callback(anio_indicador, 'RANGO_EDAD', 7, 6, 'NRO_PERSONAS')
dfEdad2016 = map_barrios(dfEdad2016, dfBarrios)
dfEdad2016.to_excel("../../data_preprocesada_N1/edad_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfEdad2018 = iteracion_callback(anio_indicador, 'RANGO_EDAD', 9, 6, 'NRO_PERSONAS')
dfEdad2018 = map_barrios(dfEdad2018, dfBarrios)
dfEdad2018.to_excel("../../data_preprocesada_N1/edad_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2019
dfEdad2019 = iteracion_callback(anio_indicador, 'RANGO_EDAD', 9, 6, 'NRO_PERSONAS')
dfEdad2019 = map_barrios(dfEdad2019, dfBarrios)
dfEdad2019.to_excel("../../data_preprocesada_N1/edad_población_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfEdad2016, dfEdad2018, dfEdad2019]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/edad_barrios_madrid.xlsx",  index=False)


/Users/renangfz/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


## Renta neta media anual de la población periodo 2016, 2018, 2019

In [57]:
# Dataset Nivel 1
anio_indicador = 2016
dfRenta2016 = iteracion_callback(anio_indicador, 'INDICADOR', 40, 1, 'RENTA_MEDIA')
dfRenta2016 = map_barrios(dfRenta2016, dfBarrios)
dfRenta2016.to_excel("../../data_preprocesada_N1/renta_neta_media_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfRenta2018 = iteracion_callback(anio_indicador, 'INDICADOR', 53, 1, 'RENTA_MEDIA')
dfRenta2018 = map_barrios(dfRenta2018, dfBarrios)
dfRenta2018.to_excel("../../data_preprocesada_N1/renta_neta_media_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2019
dfRenta2019 = iteracion_callback(anio_indicador, 'INDICADOR', 53, 1, 'RENTA_MEDIA')
dfRenta2019 = map_barrios(dfRenta2019, dfBarrios)
dfRenta2019.to_excel("../../data_preprocesada_N1/renta_neta_media_población_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfRenta2016, dfRenta2018, dfRenta2019]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/renta_neta_media_barrios_madrid.xlsx", index=False)


## Nivel Educación población periodo 2016, 2018, 2019

In [58]:
# Dataset Nivel 1
anio_indicador = 2016
dfNivelEducacion2016 = iteracion_callback(anio_indicador, 'NIVELES_EDUCACION', 98, 5, 'NRO_PERSONAS')
dfNivelEducacion2016 = map_barrios(dfNivelEducacion2016, dfBarrios)
dfNivelEducacion2016.to_excel("../../data_preprocesada_N1/nivel_educación_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfNivelEducacion2018 = iteracion_callback(anio_indicador, 'NIVELES_EDUCACION', 113, 6, 'NRO_PERSONAS')
dfNivelEducacion2018 = map_barrios(dfNivelEducacion2018, dfBarrios)
dfNivelEducacion2018.to_excel("../../data_preprocesada_N1/nivel_educación_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2019
dfNivelEducacion2019 = iteracion_callback(anio_indicador, 'NIVELES_EDUCACION', 113, 6, 'NRO_PERSONAS')
dfNivelEducacion2019 = map_barrios(dfNivelEducacion2019, dfBarrios)
dfNivelEducacion2019.to_excel("../../data_preprocesada_N1/nivel_educación_población_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfNivelEducacion2016, dfNivelEducacion2018, dfNivelEducacion2019]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/nivel_educación_barrios_madrid.xlsx", index=False)


## Número de Extranjeros 2016, 2018, 2019

In [59]:
# Dataset Nivel 1
anio_indicador = 2016
dfNroExtranjeros2016 = iteracion_callback(anio_indicador, 'INDICADOR', 20, 1, 'NRO_PERSONAS')
dfNroExtranjeros2016 = map_barrios(dfNroExtranjeros2016, dfBarrios)
dfNroExtranjeros2016.to_excel("../../data_preprocesada_N1/nro_extranjeros_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfNroExtranjeros2018 = iteracion_callback(anio_indicador, 'INDICADOR', 27, 1, 'NRO_PERSONAS')
dfNroExtranjeros2018 = map_barrios(dfNroExtranjeros2018, dfBarrios)
dfNroExtranjeros2018.to_excel("../../data_preprocesada_N1/nro_extranjeros_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2019
dfNroExtranjeros2019 = iteracion_callback(anio_indicador, 'INDICADOR', 27, 1, 'NRO_PERSONAS')
dfNroExtranjeros2019 = map_barrios(dfNroExtranjeros2019, dfBarrios)
dfNroExtranjeros2019.to_excel("../../data_preprocesada_N1/nro_extranjeros_población_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfNroExtranjeros2016, dfNroExtranjeros2018, dfNroExtranjeros2019]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/nro_extranjeros_barrios_madrid.xlsx", index=False)

## Tamaño medio del hogar 2016, 2018, 2019

In [60]:
# Dataset Nivel 1
anio_indicador = 2016
dfSizeHogar2016 = iteracion_callback(anio_indicador, 'INDICADOR', 27, 1, 'NRO_MEDIA_PERSONAS')
dfSizeHogar2016 = map_barrios(dfSizeHogar2016, dfBarrios)
dfSizeHogar2016.to_excel("../../data_preprocesada_N1/size_medio_hogares_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfSizeHogar2018 = iteracion_callback(anio_indicador, 'INDICADOR', 35, 1, 'NRO_MEDIA_PERSONAS')
dfSizeHogar2018 = map_barrios(dfSizeHogar2018, dfBarrios)
dfSizeHogar2018.to_excel("../../data_preprocesada_N1/size_medio_hogares_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2019
dfSizeHogar2019 = iteracion_callback(anio_indicador, 'INDICADOR', 35, 1, 'NRO_MEDIA_PERSONAS')
dfSizeHogar2019 = map_barrios(dfSizeHogar2019, dfBarrios)
dfSizeHogar2019.to_excel("../../data_preprocesada_N1/size_medio_hogares_población_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfSizeHogar2016, dfSizeHogar2018, dfSizeHogar2019]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/size_medio_hogares_barrios_madrid.xlsx", index=False)


## Número de desempleados 2016, 2018, 2019

In [61]:
# Dataset Nivel 1
anio_indicador = 2016
dfDesempleados2016 = iteracion_callback(anio_indicador, 'INDICADOR', 49, 1, 'NRO_PERSONAS')
dfDesempleados2016 = map_barrios(dfDesempleados2016, dfBarrios)
dfDesempleados2016.to_excel("../../data_preprocesada_N1/desempleados_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfDesempleados2018 = iteracion_callback(anio_indicador, 'INDICADOR', 65, 1, 'NRO_PERSONAS')
dfDesempleados2018 = map_barrios(dfDesempleados2018, dfBarrios)
dfDesempleados2018.to_excel("../../data_preprocesada_N1/desempleados_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2019
dfDesempleados2019 = iteracion_callback(anio_indicador, 'INDICADOR', 65, 1, 'NRO_PERSONAS')
dfDesempleados2019 = map_barrios(dfDesempleados2019, dfBarrios)
dfDesempleados2019.to_excel("../../data_preprocesada_N1/desempleados_población_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfDesempleados2016, dfDesempleados2018, dfDesempleados2019]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/desempleados_barrios_madrid.xlsx", index=False)


## Precio de la Vivienda (Nuevas, Usadas) 2016, 2018, 2019

Continuando con la integración de los datos, en la siguiente URL se extraerán los valores del indicador **Precio de la Vivienda**

> http://www-2.munimadrid.es/CSE6/control/seleccionDatos?numSerie=5040201060

In [62]:
#Función para lectura iterativa de excel Compra-Venta Vivienda
def iteracion_callback_f2(df, year, indicadorName, unidad):
    dfPreProcesadoFull = pd.DataFrame()
    i = 0
    contador_x = 0
    contador_y = 172
    while i < 1:
        # Inicializa dataframes
        dfAux = pd.DataFrame()
        # Inicializa iteradores de filas
        contadorX = contador_x + 1
        contadorY = contadorX + contador_y        
        dfAux = df[contadorX:contadorY]
        dfAux.reset_index(inplace=True, drop=True)
        # Pasar columnas como valores del dataframe bajo el campo Barrios
        df2 = pd.melt(dfAux, id_vars=[indicadorName], 
                      var_name="TIPO_VIVIENDA", value_name=unidad)        
        # Agrega columna anio
        df2['ANIO'] = year
        # Agrega columna Indicador
        df2['INDICADOR'] = 'PRECIO COMPRA VIVIENDA'        
        auxBarrio = df2['BARRIOS'].str.split('\. ', 1, expand=True)
        df2['BARRIOS'] = auxBarrio[1]
        # Strings a mayúsculas
        df2 = df2.apply(lambda x: x.str.upper() if x.dtype == "object" else x)  
        #Contadores
        contador_x = contadorY        
        i = i + 1
    df2.drop(df2[df2['PRECIO(EUROS/M2)'] == '-' ].index, inplace = True)
    # Reemplazamos barrios mal escritos
    df2.loc[df2['BARRIOS'] == 'LOS JERÓNIMOS', 'BARRIOS'] = 'JERÓNIMOS'
    df2.loc[df2['BARRIOS'] == 'FUENTELARREINA', 'BARRIOS'] = 'FUENTELAREINA'
    df2.loc[df2['BARRIOS'] == 'ÁGUILAS', 'BARRIOS'] = 'LAS ÁGUILAS'        
    df2.loc[df2['BARRIOS'] == 'ÁNGELES', 'BARRIOS'] = 'LOS ÁNGELES'        
    df2.loc[df2['BARRIOS'] == 'EL SALVADOR', 'BARRIOS'] = 'SALVADOR'        
    return df2

In [63]:
#Lectura de excel precios vivienda 2016.xls
df2016 = pd.read_excel('../../original_data/precio_viviendas_barrios_2016.xls', 
                   sheet_name=0, 
                   skiprows= 7 ,
                   usecols="A,B,C")

#Lectura de excel precios vivienda 2018.xls
df2018 = pd.read_excel('../../original_data/precio_viviendas_barrios_2018.xls', 
                   sheet_name=0, 
                   skiprows= 7 ,
                   usecols="A,B,C")

#Lectura de excel precios vivienda 2020.xls
df2019 = pd.read_excel('../../original_data/precio_viviendas_barrios_2019.xls', 
                   sheet_name=0, 
                   skiprows= 7 ,
                   usecols="A,B,C")

# Dataset Nivel 1
anio_indicador = 2016
dfPrecioVivienda2016 = iteracion_callback_f2(df2016, anio_indicador, 'BARRIOS', 'PRECIO(EUROS/M2)')
dfPrecioVivienda2016 = map_barrios(dfPrecioVivienda2016, dfBarrios)
dfPrecioVivienda2016.to_excel("../../data_preprocesada_N1/precio_vivienda_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfPrecioVivienda2018 = iteracion_callback_f2(df2018, anio_indicador, 'BARRIOS', 'PRECIO(EUROS/M2)')
dfPrecioVivienda2018 = map_barrios(dfPrecioVivienda2018, dfBarrios)
dfPrecioVivienda2018.to_excel("../../data_preprocesada_N1/precio_vivienda_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2019
dfPrecioVivienda2019 = iteracion_callback_f2(df2019, anio_indicador, 'BARRIOS', 'PRECIO(EUROS/M2)')
dfPrecioVivienda2019 = map_barrios(dfPrecioVivienda2019, dfBarrios)
dfPrecioVivienda2019.to_excel("../../data_preprocesada_N1/precio_vivienda_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfPrecioVivienda2016, dfPrecioVivienda2018, dfPrecioVivienda2019]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/precio_vivienda_barrios_madrid.xlsx", index=False)


## Turismo (Hospedaje) (Nuevas, Usadas) 2016, 2018, 2019

Con la siguiente URL se extraerán los valores correspondientes al indicador **Turismo - Hospedaje**

> https://www.kaggle.com/code/rusiano/housing-crisis-and-tourism-in-madrid/data?select=listings.csv

In [64]:
# Lectura del CSV de Airbnb
dfHospedaje = pd.read_csv('../../original_data/airbnb_turismo_barrios.csv')
dfHospedaje['ANIO'] = pd.to_datetime(dfHospedaje['last_review']).dt.year
dfHospedaje = dfHospedaje[['id','neighbourhood','latitude', 'longitude', 'room_type', 'ANIO']]

dfHospedaje = dfHospedaje.rename(columns = {'id': 'ID',
                                  'neighbourhood': 'BARRIOS',
                                  'latitude': 'LATITUD_H',
                                  'longitude': 'LONGITUD_H',
                                  'room_type': 'CATEGORIA'}, inplace = False)
# Todo mayuscula
dfHospedaje['BARRIOS'] = dfHospedaje['BARRIOS'].str.upper()
dfHospedaje['CATEGORIA'] = dfHospedaje['CATEGORIA'].str.upper()

# Elimina valores NA en el campo fecha
dfHospedaje = dfHospedaje.dropna()

# Filtra anios desde el 2018
dfHospedaje = dfHospedaje[dfHospedaje['ANIO'].isin([2016, 2018, 2019])]

# Verifica datos duplicados
dfHospedajeDeleteDuplicados = dfHospedaje[dfHospedaje.duplicated(['LATITUD_H', 'LONGITUD_H'])]

# Elimina datos duplicados
dfHospedaje.drop(dfHospedaje[dfHospedaje['ID'] == 17453051 ].index, inplace = True)
dfHospedaje.drop(dfHospedaje[dfHospedaje['ID'] == 17809905 ].index, inplace = True)
dfHospedaje.drop(dfHospedaje[dfHospedaje['ID'] == 20011461 ].index, inplace = True)
dfHospedaje.drop(dfHospedaje[dfHospedaje['ID'] == 22312525 ].index, inplace = True)
dfHospedaje.drop(dfHospedaje[dfHospedaje['ID'] == 23576224 ].index, inplace = True)
dfHospedaje.drop(dfHospedaje[dfHospedaje['ID'] == 35217403 ].index, inplace = True)
dfHospedaje.drop(dfHospedaje[dfHospedaje['ID'] == 38310467 ].index, inplace = True)

# Reemplazamos barrios mal escritos
dfHospedaje.loc[dfHospedaje['BARRIOS'] == 'PUERTA DEL ANGEL', 'BARRIOS'] = 'PUERTA DEL ÁNGEL'
dfHospedaje.loc[dfHospedaje['BARRIOS'] == 'RIOS ROSAS', 'BARRIOS'] = 'RÍOS ROSAS'
dfHospedaje.loc[dfHospedaje['BARRIOS'] == 'CARMENES', 'BARRIOS'] = 'LOS CÁRMENES'
dfHospedaje.loc[dfHospedaje['BARRIOS'] == 'LOS ANGELES', 'BARRIOS'] = 'LOS ÁNGELES'
dfHospedaje.loc[dfHospedaje['BARRIOS'] == 'AGUILAS', 'BARRIOS'] = 'LAS ÁGUILAS'
dfHospedaje.loc[dfHospedaje['BARRIOS'] == 'SAN CRISTÓBAL', 'BARRIOS'] = 'SAN CRISTÓBAL'
dfHospedaje.loc[dfHospedaje['BARRIOS'] == 'APOSTOL SANTIAGO', 'BARRIOS'] = 'APÓSTOL SANTIAGO'



In [65]:
# Dataset Nivel 1
anio_indicador = 2016
dfHospedaje2016 = dfHospedaje[dfHospedaje['ANIO'].isin([anio_indicador])]
dfHospedaje2016 = map_barrios(dfHospedaje2016, dfBarrios)
dfHospedaje2016.to_excel("../../data_preprocesada_N1/hospedaje_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfHospedaje2018 = dfHospedaje[dfHospedaje['ANIO'].isin([anio_indicador])]
dfHospedaje2018 = map_barrios(dfHospedaje2018, dfBarrios)
dfHospedaje2018.to_excel("../../data_preprocesada_N1/hospedaje_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2019
dfHospedaje2019 = dfHospedaje[dfHospedaje['ANIO'].isin([anio_indicador])]
# Reemplazamos barrios mal escritos
dfHospedaje2019.loc[dfHospedaje2019['BARRIOS'] == 'SAN CRISTOBAL', 'BARRIOS'] = 'SAN CRISTÓBAL'
dfHospedaje2019.loc[dfHospedaje2019['BARRIOS'] == 'CÁRMENES', 'BARRIOS'] = 'LOS CÁRMENES'
dfHospedaje2019 = map_barrios(dfHospedaje2019, dfBarrios)
dfHospedaje2019.to_excel("../../data_preprocesada_N1/hospedaje_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfHospedaje2016, dfHospedaje2018, dfHospedaje2019]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/hospedaje_barrios_madrid.xlsx", index=False)


/Users/renangfz/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## Terrazas 2016, 2018, 2019

Con la siguiente URL se extraerán los valores correspondientes al indicador **Número de Terrazas**

> https://datos.madrid.es/sites/v/index.jsp?vgnextoid=374512b9ace9f310VgnVCM100000171f5a0aRCRD&buscar=true&Texto=terrazas&Sector=&Formato=&Periodicidad=&orderByCombo=CONTENT_INSTANCE_NAME_DECODE

In [70]:
def replaceNameBarrios(dataF) :
    # Reemplazamos barrios mal escritos
    dataF.loc[dataF['BARRIOS'] == 'PE√ëA GRANDE', 'BARRIOS'] = 'PEÑAGRANDE'
    dataF.loc[dataF['BARRIOS'] == 'ARGUELLES', 'BARRIOS'] = 'ARGÜELLES'
    dataF.loc[dataF['BARRIOS'] == 'CASCO H.BARAJAS', 'BARRIOS'] = 'CASCO HISTÓRICO DE BARAJAS'
    dataF.loc[dataF['BARRIOS'] == 'CASCO H.VALLECAS', 'BARRIOS'] = 'CASCO HISTÓRICO DE VALLECAS'
    dataF.loc[dataF['BARRIOS'] == 'CASCO H.VICALVARO', 'BARRIOS'] = 'CASCO HISTÓRICO DE VICÁLVARO'
    dataF.loc[dataF['BARRIOS'] == 'CIUDAD JARDIN', 'BARRIOS'] = 'CIUDAD JARDÍN'
    dataF.loc[dataF['BARRIOS'] == 'CONCEPCION', 'BARRIOS'] = 'CONCEPCIÓN'
    dataF.loc[dataF['BARRIOS'] == 'EL PILAR', 'BARRIOS'] = 'PILAR'
    dataF.loc[dataF['BARRIOS'] == 'EL SALVADOR', 'BARRIOS'] = 'SALVADOR'
    dataF.loc[dataF['BARRIOS'] == 'ENTREVIAS', 'BARRIOS'] = 'ENTREVÍAS'
    dataF.loc[dataF['BARRIOS'] == 'FONTARRON', 'BARRIOS'] = 'FONTARRÓN'
    dataF.loc[dataF['BARRIOS'] == 'HELLIN', 'BARRIOS'] = 'HELLÍN'
    dataF.loc[dataF['BARRIOS'] == 'HISPANOAMERICA', 'BARRIOS'] = 'HISPANOAMÉRICA'    
    dataF.loc[dataF['BARRIOS'] == 'LAS AGUILAS', 'BARRIOS'] = 'LAS ÁGUILAS'    
    dataF.loc[dataF['BARRIOS'] == 'LOS ANGELES', 'BARRIOS'] = 'LOS ÁNGELES'        
    dataF.loc[dataF['BARRIOS'] == 'LOS CARMENES', 'BARRIOS'] = 'LOS CÁRMENES'            
    dataF.loc[dataF['BARRIOS'] == 'LOS JERONIMOS', 'BARRIOS'] = 'JERÓNIMOS'            
    dataF.loc[dataF['BARRIOS'] == 'MOSCARDO', 'BARRIOS'] = 'MOSCARDÓ'                
    dataF.loc[dataF['BARRIOS'] == 'NI√ëO JESUS', 'BARRIOS'] = 'NIÑO JESÚS'                    
    dataF.loc[dataF['BARRIOS'] == 'NUEVA ESPA√ëA', 'BARRIOS'] = 'NUEVA ESPAÑA'                    
    dataF.loc[dataF['BARRIOS'] == 'OPA√ëEL', 'BARRIOS'] = 'OPAÑEL'                        
    dataF.loc[dataF['BARRIOS'] == 'PACIFICO', 'BARRIOS'] = 'PACÍFICO'                            
    dataF.loc[dataF['BARRIOS'] == 'PUERTA DEL ANGEL', 'BARRIOS'] = 'PUERTA DEL ÁNGEL'                                
    dataF.loc[dataF['BARRIOS'] == 'RIOS ROSAS', 'BARRIOS'] = 'RÍOS ROSAS'                                    
    dataF.loc[dataF['BARRIOS'] == 'SAN ANDRES', 'BARRIOS'] = 'SAN ANDRÉS'                                        
    dataF.loc[dataF['BARRIOS'] == 'SAN CRISTOBAL', 'BARRIOS'] = 'SAN CRISTÓBAL'
    dataF.loc[dataF['BARRIOS'] == 'SAN FERMIN', 'BARRIOS'] = 'SAN FERMÍN'
    dataF.loc[dataF['BARRIOS'] == 'TIMON', 'BARRIOS'] = 'TIMÓN'    
    dataF.loc[dataF['BARRIOS'] == 'VALDEMARIN', 'BARRIOS'] = 'VALDEMARÍN'        
    dataF.loc[dataF['BARRIOS'] == 'ZOFIO', 'BARRIOS'] = 'ZOFÍO'            
    return dataF


# Lectura de CSV para terrazas 2016
df2016 = pd.read_csv('../../original_data/indicador_terrazas_201612.csv', sep=';', encoding='mac_roman')
# Filtrar solo terrazas abiertas
df2016 = df2016[df2016['desc_situacion_local'].isin(['Abierto'])]
df2016 = df2016[df2016['desc_situacion_terraza'].isin(['Abierta'])]
# Seleccionar solo columnas específicas
df2016 = df2016[['desc_barrio_local', 'coordenada_x_local', 
                 'coordenada_y_local', 'rotulo']]
df2016['ANIO'] = 2016
# Renombrar columnas
df2016.rename(columns = {'desc_barrio_local':'BARRIOS', 
                         'coordenada_x_local':'LONGITUD_T', 
                         'coordenada_y_local':'LATITUD_T', 'rotulo':'ROTULO'}, inplace = True)
# Quitar espacios en blanco en nombre de barrios y terrazas
df2016['BARRIOS'] = df2016['BARRIOS'].str.strip()
df2016['ROTULO'] = df2016['ROTULO'].str.strip()

# Corrige Barrios mal escritos
df2016 = replaceNameBarrios(df2016)

# Mapea con barrios geografico
df2016 = map_barrios(df2016, dfBarrios)
df2016.to_excel("../../data_preprocesada_N1/terrazas_" + str('2016') + ".xlsx")


In [71]:
# Lectura de CSV para terrazas 2018
df2018 = pd.read_csv('../../original_data/indicador_terrazas_201812.csv', sep=';', encoding='mac_roman')
# Filtrar solo terrazas abiertas
df2018 = df2018[df2018['desc_situacion_local'].isin(['Abierto'])]
df2018 = df2018[df2018['desc_situacion_terraza'].isin(['Abierta'])]
# Seleccionar solo columnas específicas
df2018 = df2018[['desc_barrio_local', 'coordenada_x_local', 
                 'coordenada_y_local', 'rotulo']]
df2018['ANIO'] = 2018
# Renombrar columnas
df2018.rename(columns = {'desc_barrio_local':'BARRIOS', 
                         'coordenada_x_local':'LONGITUD_T', 
                         'coordenada_y_local':'LATITUD_T', 'rotulo':'ROTULO'}, inplace = True)
# Quitar espacios en blanco en nombre de barrios y terrazas
df2018['BARRIOS'] = df2018['BARRIOS'].str.strip()
df2018['ROTULO'] = df2018['ROTULO'].str.strip()

# Corrige Barrios mal escritos
df2018 = replaceNameBarrios(df2018)

# Mapea con barrios geografico
df2018 = map_barrios(df2018, dfBarrios)
df2018.to_excel("../../data_preprocesada_N1/terrazas_" + str('2018') + ".xlsx")

In [72]:
# Lectura de CSV para terrazas 2019
df2019 = pd.read_csv('../../original_data/indicador_terrazas_201912.csv', sep=';', encoding='mac_roman')
# Filtrar solo terrazas abiertas
df2019 = df2019[df2019['desc_situacion_local'].isin(['Abierto'])]
df2019 = df2019[df2019['desc_situacion_terraza'].isin(['Abierta'])]
# Seleccionar solo columnas específicas
df2019 = df2019[['desc_barrio_local', 'coordenada_x_local', 
                 'coordenada_y_local', 'rotulo']]
df2019['ANIO'] = 2019
# Renombrar columnas
df2019.rename(columns = {'desc_barrio_local':'BARRIOS', 
                         'coordenada_x_local':'LONGITUD_T', 
                         'coordenada_y_local':'LATITUD_T', 'rotulo':'ROTULO'}, inplace = True)
# Quitar espacios en blanco en nombre de barrios y terrazas
df2019['BARRIOS'] = df2019['BARRIOS'].str.strip()
df2019['ROTULO'] = df2019['ROTULO'].str.strip()

# Corrige Barrios mal escritos
df2019 = replaceNameBarrios(df2019)

# Mapea con barrios geografico
df2019 = map_barrios(df2019, dfBarrios)
df2019.to_excel("../../data_preprocesada_N1/terrazas_" + str('2019') + ".xlsx")

In [73]:
# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [df2016, df2018, df2019]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/terrazas_barrios_madrid.xlsx", index=False)
